<a href="https://www.kaggle.com/code/hilalalpak/cltv-prediction-on-flo-dataset?scriptVersionId=179818503" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install lifetimes
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("/kaggle/input/flo-data-20k/flo_data_20k.csv")
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


In [3]:
##Get the necessary functions to suppress outliers

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit)

In [4]:
replace_with_thresholds(df, "order_num_total_ever_online")
replace_with_thresholds(df, "order_num_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_online")
replace_with_thresholds(df, "customer_value_total_ever_offline")

> Alternative; columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
for col in columns:
    replace_with_thresholds(df, col)

In [5]:
df["TotalPrice"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df["TotalOrder"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,TotalPrice,TotalOrder
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],939.3700,5.0000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",2013.5500,21.0000
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",585.3200,5.0000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",121.9700,2.0000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],209.9800,2.0000


In [6]:
##The type information of time variables is entered as object.Convert this to datetime.

date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(lambda x: pd.to_datetime(x, dayfirst=True))

/tmp/ipykernel_19/88918443.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[date_columns] = df[date_columns].apply(lambda x: pd.to_datetime(x, dayfirst=True))
/tmp/ipykernel_19/88918443.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[date_columns] = df[date_columns].apply(lambda x: pd.to_datetime(x, dayfirst=True))
/tmp/ipykernel_19/88918443.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[date_columns] = df[date_columns].apply(lambda x: pd.to_datetime(x, dayfirst=True))
/tmp/ipykernel_19/88918443.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[date_column

In [7]:
df["last_order_date"].max() #2021-05-30
today_date = dt.datetime(year=2021, month=6, day=2)

Timestamp('2021-05-30 00:00:00')

In [8]:
#Creating CLTV

cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
cltv_df["T_weekly"] = (today_date - df["first_order_date"]).dt.days / 7
cltv_df["frequency"] = df["TotalOrder"]
cltv_df["monetary_cltv_avg"] = df["TotalPrice"] / df["TotalOrder"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.7143,5.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,225.0000,21.0000,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,79.0000,5.0000,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,21.0000,2.0000,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.5714,2.0000,104.9900


In [9]:
##BG/NBD, Gamma-gamma models

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.40, b: 0.00, r: 3.67>

In [10]:
##Case1

cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.7143,5.0000,187.8740,0.9711
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,225.0000,21.0000,95.8833,0.9821
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,79.0000,5.0000,117.0640,0.6693
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,21.0000,2.0000,60.9850,0.6983
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.5714,2.0000,104.9900,0.3955


In [11]:
##Case2

cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

In [12]:
##Gamma-Gamma
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary_cltv_avg'])

cltv_df.head()

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.7143,5.0000,187.8740,0.9711,1.9422,193.6327
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,225.0000,21.0000,95.8833,0.9821,1.9643,96.6650
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,79.0000,5.0000,117.0640,0.6693,1.3387,120.9676
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,21.0000,2.0000,60.9850,0.6983,1.3967,67.3201
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.5714,2.0000,104.9900,0.3955,0.7910,114.3251


In [13]:
##CLTV for 6 months

cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,  
                                   freq="W",  
                                   discount_rate=0.01)
cltv_df.sort_values(by="cltv", ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,8.0000,4.0000,1401.8000,1.0903,2.1805,1449.0605,3315.2691
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.2857,11.0000,758.0855,1.9627,3.9253,767.3606,3160.4105
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.1429,7.0000,1106.4671,0.7210,1.4421,1127.6115,1706.1692
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.7143,16.0000,501.8737,1.5619,3.1237,506.1667,1658.9627
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.4286,52.0000,166.2246,4.6447,9.2893,166.7123,1624.8745
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.5714,8.0000,601.2262,1.2618,2.5235,611.4926,1619.0748
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.5714,2.0000,862.6900,0.7912,1.5823,923.6800,1533.5062
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.1429,17.0000,259.8653,2.7699,5.5398,262.0729,1523.3079
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.8571,31.0000,228.5300,3.0758,6.1517,229.6069,1481.9906
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.7143,3.0000,743.5867,0.8687,1.7374,778.0504,1418.2907


In [14]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_df.sort_values(by="cltv", ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,cltv,segment
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,8.0000,4.0000,1401.8000,1.0903,2.1805,1449.0605,3315.2691,A
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.2857,11.0000,758.0855,1.9627,3.9253,767.3606,3160.4105,A
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.1429,7.0000,1106.4671,0.7210,1.4421,1127.6115,1706.1692,A
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.7143,16.0000,501.8737,1.5619,3.1237,506.1667,1658.9627,A
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.4286,52.0000,166.2246,4.6447,9.2893,166.7123,1624.8745,A
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.5714,8.0000,601.2262,1.2618,2.5235,611.4926,1619.0748,A
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.5714,2.0000,862.6900,0.7912,1.5823,923.6800,1533.5062,A
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.1429,17.0000,259.8653,2.7699,5.5398,262.0729,1523.3079,A
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.8571,31.0000,228.5300,3.0758,6.1517,229.6069,1481.9906,A
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.7143,3.0000,743.5867,0.8687,1.7374,778.0504,1418.2907,A


In [15]:
cltv_df.groupby("segment").agg({"count", "sum"})
cltv_df.describe()

customer_id       recency_cltv_weekly          T_weekly        frequency       monetary_cltv_avg       exp_sales_3_month       exp_sales_6_month       expected_average_profit               cltv      
                                                       sum count                 sum count         sum count        sum count               sum count               sum count               sum count                     sum count          sum count
segment                                                                                                                                                                                                                                               
D        1854e56c-491f-11eb-806e-000d3a38a36fd6ea1074-f...  4987         692828.8571  4987 809165.2857  4987 18782.0000  4987       464521.6100  4987         2036.5922  4987         4073.1844  4987             492157.6540  4987  400182.1184  4987
C        c445e4ee-6242-11ea-9d1a-000d3a38a36fec6c382a-3...  4986         461276.0000  4986 562652.0000  4986 21918.0000  4986       627198.1276  4986         2615.7048  4986         5231.4096  4986             659437.2460  4986  688582.9367  4986
B        f431bd5a-ab7b-11e9-a2fc-000d3a38a36f69b69676-1...  4986         409649.8571  4986 501774.4286  4986 25449.0000  4986       800822.2948  4986         2992.5808  4986         5985.1615  4986             837498.6553  4986  993208.7901  4986
A        cc294636-19f0-11eb-8d74-000d3a38a36fcfbda69e-5...  4986         336275.7143  4986 412398.2857  4986 33140.0000  4986      1141072.6941  4986         3845.8735  4986         7691.7470  4986            1186918.8594  4986 1802799.7278  4986

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,cltv
count,19945.0000,19945.0000,19945.0000,19945.0000,19945.0000,19945.0000,19945.0000,19945.0000
mean,95.2635,114.6147,4.9781,152.0990,0.5761,1.1522,159.2385,194.7743
std,74.5894,74.7710,4.1209,73.5241,0.2308,0.4617,77.0190,132.5441
min,0.0000,0.8571,2.0000,22.4900,0.1493,0.2985,25.0830,12.1031
25%,50.4286,74.0000,3.0000,103.6533,0.4333,0.8666,108.7706,112.1089
50%,76.5714,93.1429,4.0000,136.9062,0.5346,1.0691,142.9166,165.2027
75%,109.4286,119.5714,6.0000,182.4500,0.6686,1.3371,190.4455,239.6207
max,433.4286,437.2857,57.0000,1401.8000,4.6447,9.2893,1449.0605,3315.2691
